In [1]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.base import ModeDataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.data.spatial.modes import AvirisDataManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from spectraclass.learn.manager import ClassificationManager, cm
from spectraclass.learn.base import LearningModel
import os
from typing import List, Union, Tuple, Optional, Dict, Callable

Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [ ]:
dm: DataManager = DataManager.initialize( "img_mgr", 'aviris' )
location = "desktop"
if location == "adapt":
    dm.modal.cache_dir = "/adapt/nobackup/projects/ilab/cache"
    dm.modal.data_dir = "/css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data/"
elif location == "desktop":
    dm.modal.cache_dir = "/Volumes/Shared/Cache"
    dm.modal.data_dir = "/Users/tpmaxwel/Development/Data/Aviris"
else: raise Exception( f"Unknown location: {location}")
    
block_size = 200
method = "aec" # "vae"
model_dims = 32
version = "v2p9"  # "v2v2" "v2p9"
month = "201707" # "201707" "201908"
preprocess = True

dm.use_model_data = True
dm.proc_type = "skl"
dm.modal.images_glob = f"ang{month}*rfl/ang*_rfl_{version}/ang*_corr_{version}_img"
TileManager.block_size = block_size
TileManager.block_index = [0,5]
AvirisDataManager.version = version
dm.modal.valid_aviris_bands =  [ [5,193], [214,283], [319,420] ]
dm.modal.model_dims = model_dims
dm.modal.reduce_method = method
dm.modal.reduce_nepoch = 3
dm.modal.reduce_focus_nepoch = 10
dm.modal.reduce_niter = 1
dm.modal.reduce_focus_ratio = 10.0
dm.modal.reduce_dropout = 0.0
dm.modal.reduce_learning_rate = 1e-4
dm.modal.refresh_model = False
dm.modal.modelkey = f"b{block_size}.{method}"


Here we load the project data and define a set of class names (with associated colors) to be used in the classification process.

In [ ]:
dm.loadCurrentProject()
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]

lm().setLabels( classes )


Here we have the option to preprocess all image files (takes a long time if not done yet).

In [ ]:
if preprocess:
    dm.prepare_inputs()

Here we have the option to preprocess all image files (takes a long time if not done yet).

In [ ]:
if preprocess:
    dm.prepare_inputs()

Here we define a custom CNN to use withing the spectraclass framework.

In [4]:
from spectraclass.learn.mlp import MLP
from spectraclass.learn.cnn import CNN2D, SpectralCNN, CNN3D

cm().nepochs = 10
cm().mid = "mlp"

mlp = MLP( 'mlp', layers = [32, 16] )
cm().addNetwork(mlp)

cnn = CNN2D( 'cnn2d', cnn_layers =  [(8,3,1)], dense_layers= [32, 16] )
cm().addNetwork(cnn)

cnn1D = SpectralCNN( 'cnn1d', cnn_layers = [(8,5,3),(8,5,3)], dense_layers= [32, 16] )
cm().addNetwork(cnn1D)

/var/folders/k6/40r5lsxs75g91q824qzfvqy40000gn/T/ipykernel_48134/1293898267.py:4: UserWarning: Config option `nfeatures` not recognized by `ClassificationManager`.
  cm().nepochs = 60
2023-01-02 15:41:56.885894: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 363)]             0         
                                                                 
 dense_3 (Dense)             (None, 182)               66248     
                                                                 
 dense_4 (Dense)             (None, 91)                16653     
                                                                 
 dense_5 (Dense)             (None, 46)                4232      
                                                                 
 dense_7 (Dense)             (None, 64)                3008      
                                                                 
 dense_8 (Dense)             (None, 128)               8320      
                                                                 
 dense_9 (Dense)             (None, 256)               33024 

Here we start up the Spectraclass GUI.

In [5]:
controller: SpectraclassController = app()
controller.gui()

Initializing GUI using controller <class 'spectraclass.gui.spatial.application.Spectraclass'>
UMAP.init: model_data('samples', 'band') shape = (36574, 32)
Get Activation flow for dsid 20170720t004130-0-5
ActivationFlow.instance: shape=(36574, 32), nn=5, args={'metric': 'cosine', 'p': 2}
 --->  $$$D: setNodeData D=> <class 'numpy.ndarray'>:float32
Computed NN skGraph with 5 neighbors and 36574 verts in 26.69175910949707 sec (0.44486265182495116 min)
 --->  $$$D: setNodeData D=> <class 'numpy.ndarray'>:float32
Computed NN skGraph with 5 neighbors and 36574 verts in 25.869584798812866 sec (0.4311597466468811 min)
Get Activation flow for dsid 20170720t004130-0-5
Get Activation flow for dsid 20170720t004130-0-5
Computing umap embedding with nepochs = 1, alpha = 1.0, nLabels = 0, n_neighbors = 5, init = random (random)
Completed UMAP random initialization, init shape = (36574, 3)
Completed simplicial_set_embedding, times = 0.0004497369130452474 6.429354349772135e-06 min
umap embedding comple